In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import sys
!{sys.executable} -m pip install google-cloud-bigquery
from google.cloud import bigquery

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BigqueryExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.8/183.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 10.6 MB/s eta 0:00:00


# Load tables

In [3]:
from pyspark.sql.functions import col
df_mortage = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.mortage_amount_batch")    # project_id.datatset.tablename. Use your project id
df_mortage = df_mortage.withColumn("possible_mortgage_amount", col("possible_mortgage_amount").cast("int"))

df_mortage.printSchema()

root
 |-- ID: long (nullable = true)
 |-- spouse_ID: long (nullable = true)
 |-- max_to_spend_month: double (nullable = true)
 |-- possible_mortgage_amount: integer (nullable = true)



In [4]:
df_housing = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.house_pricing")    # project_id.datatset.tablename. Use your project id
df_housing = df_housing.withColumn("Price", col("Price").cast("int"))
df_housing.printSchema()


root
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Lot_size: string (nullable = true)
 |-- Living_space_size: string (nullable = true)
 |-- Build_year: string (nullable = true)
 |-- Build_type: string (nullable = true)
 |-- House_type: string (nullable = true)
 |-- Roof: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Toilet: string (nullable = true)
 |-- Floors: string (nullable = true)
 |-- Energy_label: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Garden: string (nullable = true)
 |-- Estimated_neighbourhood_price_per: double (nullable = true)
 |-- Availability: boolean (nullable = true)
 |-- event_time: timestamp (nullable = true)



## Take a random person from the dataset

In [30]:
from pyspark.sql.functions import col, udf, window
from random import randint

def select_person_id():
    chosen_ID = randint(1, df_mortage.count())
    return chosen_ID

selected_df = df_mortage.filter( col("ID") == select_person_id())
selected_df.show()

+---+---------+------------------+------------------------+
| ID|spouse_ID|max_to_spend_month|possible_mortgage_amount|
+---+---------+------------------+------------------------+
|855|      855|          2503.635|                  497615|
+---+---------+------------------+------------------------+



## Take all the houses that that have a lower price than the possible mortage amount and are available

In [31]:
available_houses_chosen_individual = df_housing.filter(col("Price") < selected_df.first()\
                                                       .asDict()["possible_mortgage_amount"])\
                                                        .filter(col("Availability") == True)

available_houses = available_houses_chosen_individual.select('Address','City','Price','Availability',"event_time").orderBy(col("Price").desc())
available_houses.show(10)


+--------------------+----------+------+------------+--------------------+
|             Address|      City| Price|Availability|          event_time|
+--------------------+----------+------+------------+--------------------+
|Koningin Wilhelmi...|    Houten|497000|        true|2023-12-01 02:50:...|
|       Lazuursteen 9|    Houten|495000|        true|2023-12-01 00:00:...|
|     Monnickskamp 77|    Huizen|495000|        true|2023-12-01 02:22:...|
|Wethouder Roodenb...|   Haarlem|495000|        true|2023-12-01 03:04:...|
|     Weberstraat 144|Amersfoort|495000|        true|2023-12-01 01:16:...|
| Wieger Bruinlaan 34| Hoofddorp|495000|        true|2023-12-01 00:04:...|
|       Hanzestraat 7|Maasbommel|495000|        true|2023-12-01 03:31:...|
|   Stationsstraat 35|Roosendaal|495000|        true|2023-11-30 20:28:...|
|    Waltmanstraat 49|Zwaanshoek|495000|        true|2023-12-01 01:43:...|
|       Galjoen 19 12|  Lelystad|495000|        true|2023-12-01 02:16:...|
+--------------------+---

In [32]:
selected_houses = spark.createDataFrame(available_houses.take(500))
selected_houses.count()

500

In [33]:
# Initialize the BigQuery client
client = bigquery.Client(project="degroup11")  

# Define the table schema
schema = [
    bigquery.SchemaField("Address", "STRING"),
    bigquery.SchemaField("City", "STRING"),
    bigquery.SchemaField("Price", "STRING"),
    bigquery.SchemaField("Availability", "BOOLEAN"),
    bigquery.SchemaField("event_time", "TIMESTAMP"),

]

# Specify the BigQuery dataset and table
dataset_id = "group11dataset"  
table_id = "available_houses_for_individual"   

# Create the BigQuery table
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# Create the table if it doesn't exist
client.create_table(table, exists_ok=True)

Table(TableReference(DatasetReference('degroup11', 'group11dataset'), 'available_houses_for_individual'))

In [34]:
selected_houses.write.format("bigquery").\
option('table', "degroup11.group11dataset.available_houses_for_individual").\
option("temporaryGcsBucket", "tomporay_bucket"). \
mode("overwrite").save()